In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from scipy.sparse import hstack, csr_matrix
import joblib

In [ ]:
df=pd.read_txt()

In [ ]:
df=pd.read_json(r"E:\Phising_detection\dataset\urls.json")
df.shape

In [ ]:
# Load your dataset
# df = pd.read_csv(r"E:\Phising_detection\dataset\PhiUSIIL_Phishing_URL_Dataset.csv")
df.dropna(inplace=True)
df.drop_duplicates(inplace=True)
# Check the first few rows
print(df.head())
print(df.columns)

In [ ]:
df['label']

In [ ]:
df.shape

In [ ]:
# Prepare TF-IDF from raw URL string
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(3, 6), max_features=3000)
X_tfidf = vectorizer.fit_transform(df['text'].astype(str))

In [ ]:
df['label']

In [ ]:
# Combine both inputs
# X_combined = hstack([X_tfidf, csr_matrix(X_numeric_scaled)])
y = df['label']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, y, test_size=0.2, stratify=y, random_state=42
)

In [ ]:
# Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:


# Save Random Forest model
joblib.dump(clf, "url_model.pkl")

# Save TF-IDF vectorizer
joblib.dump(vectorizer, "vectorizer.pkl")

# Save feature scaler
# joblib.dump(scaler, "feature_scaler.pkl")


In [ ]:
import numpy as np
from scipy.sparse import hstack

def predict_url_phishing(url):
    # Load saved objects
    clf = joblib.load("url_model.pkl")
    vectorizer = joblib.load("vectorizer.pkl")
 
    # TF-IDF transform
    tfidf_vector = vectorizer.transform([url])

    prediction = clf.predict(tfidf_vector)[0]
    probability = clf.predict_proba(tfidf_vector)[0][1]

    return prediction, round(probability, 4)


In [ ]:
url = "https://chatgpt.com/c/68762bb9-04cc-800d-853f-b119f84ea6f8"
pred, prob = predict_url_phishing(url)
print(pred,prob)
print(f"Prediction: {'Phishing' if pred==0 else 'Legitimate'} (Confidence: {prob})")
# Label 1 corresponds to a legitimate URL, label 0 to a phishing URL